In [1]:
import heapq
import numpy as np
from collections import defaultdict
import time

In [2]:
# DANGER
#with open("tests/test3_new.txt") as f:
#    test3 = f.read().splitlines()

In [56]:
class Vertex:

    def __init__(self, id, weight):
        self.key = 0
        self.id = id
        self.visited = False
        self.weight = weight
        self.neighbour_count = 0
        self.neighbour_weight = 0
        self.distance = 0
        self.sum = 0
        self.neighbour_neighbour_count = 0
        self.neighbour_neighbour_weight = 0
        self.neighbour_distance = 0
        self.neighbour_sum = 0
        self.neighbour_list = []
        
    def add_neighbour(self, Vertex, distance):
        self.neighbour_list += [Vertex]
        self.neighbour_count = self.neighbour_count + 1
        self.distance = self.distance + distance
        self.neighbour_weight = self.neighbour_weight + Vertex.weight
        self.sum = self.sum + distance *  Vertex.weight
        self.visited = True
        
    def add_neighbour_neighbour(self, Vertex):
        self.neighbour_neighbour_count +=  Vertex.neighbour_count
        self.neighbour_neighbour_weight += Vertex.neighbour_weight
        self.neighbour_distance +=  Vertex.distance
        self.neighbour_sum += Vertex.sum
        self.visited = True

    def calculate(self, a1, a2, a3):
        self.key = self.weight *  a1 +  self.sum *  a2 +  self.neighbour_sum * a3   
        return self.key

    def get_key(self):
        return self.key
    
    def __str__(self):
        return "Key: " + self.key.__str__() +  " Id: " + self.id.__str__() + \
        " Visited: " + self.visited.__str__() + " Weight: " + self.weight.__str__() +  \
        " Neighbour_count: " + self.neighbour_count.__str__() + " Neighbour_weight: " + self.neighbour_weight.__str__() + \
        " Distance: " + self.distance.__str__() + " Sum: " + self.sum.__str__() + \
        " N_N_count: " + self.neighbour_neighbour_count.__str__() + " N_N_weight: " + self.neighbour_neighbour_weight.__str__() + \
        " N_Distance: " + self.neighbour_distance.__str__() + " N_Sum: " + self.neighbour_sum.__str__() +  "\n"
    

In [57]:
with open("tests/test2_new.txt") as f:
    lines = f.read().splitlines()

number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(ids_and_populations)

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight

In [5]:
vertex_list = []

for i in range(len(populations)):
    v = Vertex(i, populations[i])
    vertex_list += [v] 

for i in range(len(edges)):
    index1, index2, distance = map(int, edges[i].split(" "))
    v0 = vertex_list[index1]
    v1 = vertex_list[index2]
    v0.add_neighbour(v1, distance)
    v1.add_neighbour(v0, distance)
    

for i in range(len(vertex_list)):
    Vlist = vertex_list[i].neighbour_list
    v0 = vertex_list[i]
    for j in range(len(Vlist)):
        v0.add_neighbour_neighbour(vertex_list[j])

In [58]:
def dijkstra(G, source):
    start_time1 = time.time()
    costs = dict()
    for i in range(number_of_vertices):
        costs[i] = np.inf
    costs[source] = 0
    #print(f"Costs dict: {time.time() - start_time1}")
    
    #start_time = time.time()
    pq = []
    
    for node in G:
        heapq.heappush(pq, (node, costs[node]))
    #print(f"Heap Push: {time.time() - start_time}")
    
    #start_time = time.time()
    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
    #print(f"While Pop: {time.time() - start_time}")
    print(f"Total Time: {time.time() - start_time1} sec")
    return np.sum(np.array(list(costs.values())) * np.array(list(populations.values())))

In [7]:
def dijkstra_path(G, population_list, source):
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0
    #display(source,costs)

    pq = []
    for node in G:
        heapq.heappush(pq, (node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            #print(current_node,costs[source])
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                #if source==neighbor_node:
                    #print('here')
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
    
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    sorted_populations_lst = list(dict(sorted(population_list.items())).values())
    #print(np.array(sorted_costs_lst) ,np.array(sorted_populations_lst))
    return np.array(sorted_costs_lst) * np.array(sorted_populations_lst)
    #return list(dict(sorted(costs.items())).values())

In [8]:
#APSP matrix
#def dijkstra(G, population_dict, source):


APSP = np.zeros((number_of_vertices,number_of_vertices))
population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
for vertex in vertices:
    vertex= int(vertex.split()[0])
    APSP[vertex] = [e for e in dijkstra_path(G, population_dict,vertex)]


def V4(APSP):
    global glob
    
    n = len(APSP)
    print(n)

    res = {}

    temp_arr = APSP.copy()

    count=0
    count2=0
    for first in range(n):
        for second in range(first+1,n):
            if first==second:
                continue
            count+=1
            #print(count)
            #print(first,second)
            temp_arr = APSP.copy()
            for row in temp_arr:
                if row[first]<row[second]:
                    row[second]=0
                else:
                    row[first]=0 
            #print(temp_arr,count)
            
            #temp = sum(sum(temp_arr)) 
            to_be_summed = temp_arr[:,[first,second]]
            #display(to_be_summed)
            summed = sum(sum(to_be_summed)) 
            #glob=to_be_summed
            #if temp in res.keys():
                #print(res[sum(sum(temp_arr))])
                #print('here!!!!!!!! ' ,first,second)
            #print(temp,(first,second))
            #res[temp]=(first,second) #FIXME: there are collisions but is it a problem?
            res[(first,second)]=summed
    #print(count2)
    #print(count)

    #ret=res[min(res.keys())]
    ret=min(res, key=res.get)
    display(len(res))
    display(res)
    print('pick {}th and {}th vertices to place hospitals!'.format(ret[0],ret[1]))
    return res

#print(APSP)
answer = V4(APSP)

250


31125

{(0, 1): 9750832.0,
 (0, 2): 4005812.0,
 (0, 3): 10291016.0,
 (0, 4): 1996885.0,
 (0, 5): 475244.0,
 (0, 6): 10037951.0,
 (0, 7): 10275926.0,
 (0, 8): 10139457.0,
 (0, 9): 9734184.0,
 (0, 10): 4525562.0,
 (0, 11): 9816800.0,
 (0, 12): 8571972.0,
 (0, 13): 9463180.0,
 (0, 14): 5481440.0,
 (0, 15): 4696014.0,
 (0, 16): 5652466.0,
 (0, 17): 8281506.0,
 (0, 18): 9916112.0,
 (0, 19): 9994896.0,
 (0, 20): 9772616.0,
 (0, 21): 9923556.0,
 (0, 22): 8026882.0,
 (0, 23): 10439188.0,
 (0, 24): 1369988.0,
 (0, 25): 10210780.0,
 (0, 26): 10217986.0,
 (0, 27): 6449868.0,
 (0, 28): 7051132.0,
 (0, 29): 8321669.0,
 (0, 30): 9688448.0,
 (0, 31): 10135594.0,
 (0, 32): 10412413.0,
 (0, 33): 10444119.0,
 (0, 34): 6342122.0,
 (0, 35): 5898070.0,
 (0, 36): 9596888.0,
 (0, 37): 10187948.0,
 (0, 38): 8931164.0,
 (0, 39): 10454660.0,
 (0, 40): 9882641.0,
 (0, 41): 8765116.0,
 (0, 42): 7568456.0,
 (0, 43): 10251008.0,
 (0, 44): 10444364.0,
 (0, 45): 10412130.0,
 (0, 46): 6194831.0,
 (0, 47): 1667911.0,
 (0, 48)

pick 205th and 215th vertices to place hospitals!


In [9]:
ret=answer[min(answer.keys())]
ret_val=min(answer.keys())
    
dict2 = {}
arr = []
for (key, value) in answer.items():
    dict2[value] = key
    arr += [value]

[9750832.0, 4005812.0, 10291016.0, 1996885.0, 475244.0, 10037951.0, 10275926.0, 10139457.0, 9734184.0, 4525562.0, 9816800.0, 8571972.0, 9463180.0, 5481440.0, 4696014.0, 5652466.0, 8281506.0, 9916112.0, 9994896.0, 9772616.0, 9923556.0, 8026882.0, 10439188.0, 1369988.0, 10210780.0, 10217986.0, 6449868.0, 7051132.0, 8321669.0, 9688448.0, 10135594.0, 10412413.0, 10444119.0, 6342122.0, 5898070.0, 9596888.0, 10187948.0, 8931164.0, 10454660.0, 9882641.0, 8765116.0, 7568456.0, 10251008.0, 10444364.0, 10412130.0, 6194831.0, 1667911.0, 10400180.0, 10461811.0, 5175816.0, 9883906.0, 2382465.0, 7163522.0, 10186888.0, 10126468.0, 7383296.0, 9228197.0, 10459760.0, 10453844.0, 8219426.0, 10006080.0, 10346430.0, 10120550.0, 8613342.0, 10264478.0, 10440780.0, 10432356.0, 10451386.0, 10412228.0, 10060136.0, 10219736.0, 9417876.0, 7101769.0, 10005708.0, 9334008.0, 10210472.0, 10019084.0, 10165999.0, 5039704.0, 10459080.0, 3496178.0, 10145252.0, 10259784.0, 9836943.0, 293400.0, 10178318.0, 2408781.0, 10081

In [39]:
learning_rate = 1.0e-8
epochs = 10
bias = 1
sum = 0

def Preceptator(Vertex1, Vertex2, a1, a2, a3, output):
    outputP = Vertex1.calculate(a1, a2, a3) + Vertex2.calculate(a1, a2, a3)

    error1 = learning_rate * (output - outputP) * a1 
    error2 = learning_rate * (output - outputP) * a2
    error3 = learning_rate * (output - outputP) * a3

    a1 += error1
    a2 += error2
    a3 += error3
    return abs(output - outputP)

def getClosestValue(k):
    b = int(k)
    lst = list(range(b - 1, b + 2, 1))  # lst = [b-1,b,b+1]
    return lst[min(range(len(lst)), key=lambda i: abs(lst[i] - k))]

def Predict(Vertex1, Vertex2, a1, a2, a3):
    value = Vertex1.calculate(a1, a2, a3) + Vertex2.calculate(a1, a2, a3)
    if (value < 0):
        print("value : ", int(value) - 1)
    elif value == 0:
        print("value : ", value)
    else:
        print("value : ", int(value))
    print("y  =  ", getClosestValue(w[0]), " + ", getClosestValue(w[1]), " * x1 + ", getClosestValue(w[2]), " * x2")
    
def calculate(Vertex, a1, a2, a3, a4, a5, a6, a7, a8, a9):
    return Vertex.weight *  a1 +  Vertex.neighbour_count * a2 + \
    Vertex.neighbour_weight *  a3 +  Vertex.distance * a4 + \
    Vertex.sum *  a5 +  Vertex.neighbour_neighbour_count * a6 + \
    Vertex.neighbour_neighbour_weight *  a7 +  Vertex.neighbour_distance * a8 + \
    Vertex.neighbour_sum * a9
    
def sigmoid(X, w, w0):
    return (1 / (1 + np.exp(-(np.matmul(X, w) + w0))))


def gradient_W(X, y_truth, y_predicted):
    return (np.asarray(
        [-np.sum(np.repeat((y_truth[:, c] - y_predicted[:, c])[:, None], X.shape[1], axis=1) * X, axis=0) for c in
         range(K)]).transpose()) / 3

def gradient_w0(Y_truth, Y_predicted):
    return (-np.sum(Y_truth - Y_predicted, axis=0))

In [42]:
# training sets
#for i in range(epochs):
a1 = np.random.rand(1) * 100 
a2 = np.random.rand(1) * 10 
a3 = np.random.rand(1) * 1 
count = 0

for i in range(len(vertex_list)):
    for j in range(int(len(vertex_list))):   
        if i>j:       
            v0 = vertex_list[i]
            v1 = vertex_list[j]
            value = arr[count]
            count += 1
            for j in range(epochs):
                val = Preceptator(v0, v1, a1, a2, a3, value)
                if j == epochs-1:
                    sum += val
            
print(sum/(count)/max(arr))  #error rate = 0.07
#print(a1)   #92.72
#print(a2)   #9.87
#print(a3)   #0.89

#Normalizing the variables
coef_sum = a1 + a2 + a3
a1 = a1 / coef_sum
a2 = a2 / coef_sum
a3 = a3 / coef_sum

[0.07152036]


In [45]:
with open("tests/test3_new.txt") as f:
    lines = f.read().splitlines()

number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(ids_and_populations)

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight
    
vertex_list = []
for i in range(len(populations)):
    v = Vertex(i, populations[i])
    vertex_list += [v] 

for i in range(len(edges)):
    index1, index2, distance = map(int, edges[i].split(" "))
    v0 = vertex_list[index1]
    v1 = vertex_list[index2]
    v0.add_neighbour(v1, distance)
    v1.add_neighbour(v0, distance)
    

for i in range(len(vertex_list)):
    Vlist = vertex_list[i].neighbour_list
    v0 = vertex_list[i]
    for j in range(len(Vlist)):
        v0.add_neighbour_neighbour(vertex_list[j])

[0.30904008]
[76.16603874]
[10.88944715]
[0.25699123]
[0.87233854]
[0.12471811]
[0.00294335]


In [59]:
a1 = 92.72
a2 = 9.87
a3 = 0.89

for i in range(len(vertex_list)):    
    v0 = vertex_list[i]
    v0.calculate(a1, a2, a3)
    
K_arr = []
for i in range(len(vertex_list)):        
    K_arr += [vertex_list[i].get_key()]

min_val = min(K_arr) 
print(min_val)        #5018286599369.93
K_arr.sort()
index1 = 0
index2 = 0

for i in range(len(vertex_list)):
    v = vertex_list[i]
    if K_arr[0] == v.get_key():
        index1 = i
    if K_arr[1] == v.get_key():
        index2 = i
    
print(index1)   #73 #95760
print(index2)   #16 #907820

# 5018286599369.93
# 95760
# 907820

5018286599369.93
95760
907820


[16696, 95760, 148837, 289571, 317390, 461822, 724640, 838852, 907820, 969090]


MemoryError: 

MemoryError: 

MemoryError: 

# Algo

In [ ]:
with open("tests/test2_new.txt") as f:
    lines = f.read().splitlines()

In [ ]:
import random as r

def random_start():
    res = [r.randint(0,number_of_vertices-1),r.randint(0,number_of_vertices-1)]
    if res[0] == res [1]:
        return random_start()
    return res

#//2 * O((V+E)*logV) = O(E*logV) //
def allocation_cost(G,i,j):
    return [dijkstra(G,i),dijkstra(G,j)]


# 2*O(V)*O(E*logV) = O(E*V*logV) #
def algo(G):
    selected_vertices = random_start() #TODO np.array
    selected_costs = allocation_cost(G,selected_vertices[0],selected_vertices[1]) #TODO np.array
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra(G,not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                selected_vertices[bigger_index] = not_selected
    return(selected_vertices,selected_costs)


#%timeit algo(G)
#[algo(G) for i in range(20)]

algo(G)

In [ ]:
lst1 = [1,23,4,5,6,88]
lst2 = lst1.copy()
lst2[0] = 99
print(lst1[0]==lst2[0])

In [ ]:
import heapq
import numpy as np
from collections import defaultdict
import time
import random as r

with open("tests/test2_new.txt") as f:
    lines = f.read().splitlines()

number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(ids_and_populations)

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight

def dijkstra(G, source):
    #start_time1 = time.time()
    costs = dict()
    for i in range(number_of_vertices):
        costs[i] = np.inf
    costs[source] = 0
    #print(f"Costs dict: {time.time() - start_time1}")
    
    #start_time = time.time()
    pq = []
    
    for node in G:
        heapq.heappush(pq, (node, costs[node]))
    #print(f"Heap Push: {time.time() - start_time}")
    
    #start_time = time.time()
    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
    #print(f"While Pop: {time.time() - start_time}")
    #print(f"Total Time: {time.time() - start_time1} sec")
    return np.sum(np.array(list(costs.values())) * np.array(list(populations.values())))


    import random as r

def random_start():
    res = [r.randint(0,number_of_vertices-1),r.randint(0,number_of_vertices-1)]
    if res[0] == res [1]:
        return random_start()
    return res

#//2 * O((V+E)*logV) = O(E*logV) //
def allocation_cost(G,i,j):
    return [dijkstra(G,i),dijkstra(G,j)]


# 2*O(V)*O(E*logV) = O(E*V*logV) #
def algo(G):
    selected_vertices = random_start() #TODO np.array
    selected_costs = allocation_cost(G,selected_vertices[0],selected_vertices[1]) #TODO np.array
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra(G,not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                selected_vertices[bigger_index] = not_selected
    return(selected_vertices,selected_costs)


#%timeit algo(G)
#[algo(G) for i in range(20)]
start_time = time.time()
algo(G)
elapsed_time = time.time() - start_time

with open("output.txt", "w+") as f:
    f.write(f"Elapsed time in sec: {elapsed_time}")
f.close()